In [37]:
# Improting Geo Json
from pymongo import MongoClient
import pandas as pd

client = MongoClient('mongodb://localhost:27017/')
db = client.GeoComp
a=db.GeoComp.find()
b=pd.DataFrame(a)
b["headquarters"].head()
#b["headquarters"]["coordinates"]
b.shape

(2939, 10)

In [38]:
b.drop(columns=['_id'], inplace=True)
b.head()

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees
0,New,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.173887,...",37.480999,-122.173887,Jingle Networks,35
1,New,web,USA,2007,"{'type': 'Point', 'coordinates': [-118.487267,...",34.017606,-118.487267,Mahalo,40
2,New,web,USA,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18
3,New,web,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75


In [39]:
radio=1000
geopoint=b["headquarters"][0]
geopoint

{'type': 'Point', 'coordinates': [-122.173887, 37.480999]}

In [40]:
def findNear(geopoint,radio):
     return db.GeoComp.find({
        "headquarters": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio,
         }
       }
    })
geopoint=b["headquarters"][0]
lst=list(findNear(geopoint,radio))
lst

[{'_id': ObjectId('5d2750466497a5d8bff29a00'),
  'name': 'Jingle Networks',
  'lat': 37.480999,
  'lng': -122.173887,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-122.173887, 37.480999]},
  'number_of_employees': 35,
  'category_code': 'mobile',
  'founded_year': 2005,
  'age': 'New'},
 {'_id': ObjectId('5d2750466497a5d8bff29db0'),
  'name': 'Vivaty',
  'lat': 37.48032,
  'lng': -122.178371,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-122.178371, 37.48032]},
  'number_of_employees': 25,
  'category_code': 'web',
  'founded_year': 2007,
  'age': 'New'},
 {'_id': ObjectId('5d2750466497a5d8bff29a38'),
  'name': 'oDesk',
  'lat': 37.479952,
  'lng': -122.178557,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-122.178557, 37.479952]},
  'number_of_employees': 120,
  'category_code': 'web',
  'founded_year': 2005,
  'age': 'New'}]

In [41]:
def sum_employees(lst):
    suma=0
    for i in lst:
        suma+= i["number_of_employees"]
    return suma

n=sum_employees(lst)
n


180

In [43]:
sum_column=[]
for i in range(len(b)):
    geopoint=b["headquarters"][i]
    lost=list(findNear(geopoint,radio))
    n=sum_employees(lost)
    sum_column.append(n)


In [ ]:
#geopoint=b["headquarters"].values[i]['coordinates']

In [59]:
#https://docs.mongodb.com/manual/reference/operator/query/near/index.html

#def findNear(b, radio_max_meters=2000):

total_close=[]
for i in range(len(b)):
    coo=b["headquarters"].values[i]['coordinates']
    query = db.GEO.find({"headquarters": { 
        "$near": {"$geometry": {
            "type": "Point", 
            "coordinates": coo,
            "$maxDistance": 2000,}}}}) 
    total_close.append(list(query))

total_close
                                      
 




[[{'_id': ObjectId('5d273ea66497a5d8bff27bee'),
   'name': 'Carfeine',
   'lat': 38.989124,
   'lng': -77.026676,
   'country': 'USA',
   'headquarters': {'type': 'Point', 'coordinates': [-77.026676, 38.989124]},
   'number_of_employees': 5,
   'category_code': 'software',
   'founded_year': 2012,
   'age': 'New'},
  {'_id': ObjectId('5d273ea66497a5d8bff27bfc'),
   'name': 'CollegeConvo',
   'lat': 40.650291,
   'lng': -74.294395,
   'country': 'USA',
   'headquarters': {'type': 'Point', 'coordinates': [-74.294395, 40.650291]},
   'number_of_employees': 2,
   'category_code': 'web',
   'founded_year': 2010,
   'age': 'New'},
  {'_id': ObjectId('5d273ea66497a5d8bff27c03'),
   'name': 'eCampusCash',
   'lat': 40.650291,
   'lng': -74.294395,
   'country': 'USA',
   'headquarters': {'type': 'Point', 'coordinates': [-74.294395, 40.650291]},
   'number_of_employees': 3,
   'category_code': 'web',
   'founded_year': 2010,
   'age': 'New'},
  {'_id': ObjectId('5d273ea66497a5d8bff27bf6'),
   '

In [60]:
d=pd.DataFrame(total_close)
d

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,"{'_id': 5d273ea66497a5d8bff27bee, 'name': 'Car...","{'_id': 5d273ea66497a5d8bff27bfc, 'name': 'Col...","{'_id': 5d273ea66497a5d8bff27c03, 'name': 'eCa...","{'_id': 5d273ea66497a5d8bff27bf6, 'name': 'Int...","{'_id': 5d273ea66497a5d8bff27bf8, 'name': 'Int...","{'_id': 5d273ea66497a5d8bff27bfe, 'name': 'Yip...","{'_id': 5d273ea66497a5d8bff27bff, 'name': 'Uni...","{'_id': 5d273ea66497a5d8bff27c0a, 'name': 'Hay...","{'_id': 5d273ea66497a5d8bff27bf4, 'name': 'Jum...","{'_id': 5d273ea66497a5d8bff27c02, 'name': 'kin...",...,"{'_id': 5d273ea66497a5d8bff27c00, 'name': 'Ind...","{'_id': 5d273ea66497a5d8bff27bf0, 'name': 'Fix...","{'_id': 5d273ea66497a5d8bff27bfb, 'name': 'Sky...","{'_id': 5d273ea66497a5d8bff27c08, 'name': 'Fli...","{'_id': 5d273ea66497a5d8bff27c09, 'name': 'Rip...","{'_id': 5d273ea66497a5d8bff27c04, 'name': 'Clo...","{'_id': 5d273ea66497a5d8bff27bfa, 'name': 'Nav...","{'_id': 5d273ea66497a5d8bff27c06, 'name': 'Ten...","{'_id': 5d273ea66497a5d8bff27bf3, 'name': 'hea...","{'_id': 5d273ea66497a5d8bff27c0b, 'name': 'eki..."
1,"{'_id': 5d273ea66497a5d8bff27bef, 'name': 'GEN...","{'_id': 5d273ea66497a5d8bff27c07, 'name': 'BAS...","{'_id': 5d273ea66497a5d8bff27bf7, 'name': 'vuf...","{'_id': 5d273ea66497a5d8bff27bf9, 'name': 'vuf...","{'_id': 5d273ea66497a5d8bff27bf2, 'name': 'Zii...","{'_id': 5d273ea66497a5d8bff27bfd, 'name': 'Soc...","{'_id': 5d273ea66497a5d8bff27bf0, 'name': 'Fix...","{'_id': 5d273ea66497a5d8bff27bfb, 'name': 'Sky...","{'_id': 5d273ea66497a5d8bff27c05, 'name': 'Fuz...","{'_id': 5d273ea66497a5d8bff27c09, 'name': 'Rip...",...,"{'_id': 5d273ea66497a5d8bff27bfc, 'name': 'Col...","{'_id': 5d273ea66497a5d8bff27bf8, 'name': 'Int...","{'_id': 5d273ea66497a5d8bff27bf6, 'name': 'Int...","{'_id': 5d273ea66497a5d8bff27bfe, 'name': 'Yip...","{'_id': 5d273ea66497a5d8bff27bff, 'name': 'Uni...","{'_id': 5d273ea66497a5d8bff27c04, 'name': 'Clo...","{'_id': 5d273ea66497a5d8bff27bfa, 'name': 'Nav...","{'_id': 5d273ea66497a5d8bff27bf3, 'name': 'hea...","{'_id': 5d273ea66497a5d8bff27c06, 'name': 'Ten...","{'_id': 5d273ea66497a5d8bff27c0b, 'name': 'eki..."
2,"{'_id': 5d273ea66497a5d8bff27bf0, 'name': 'Fix...","{'_id': 5d273ea66497a5d8bff27bfb, 'name': 'Sky...","{'_id': 5d273ea66497a5d8bff27bfd, 'name': 'Soc...","{'_id': 5d273ea66497a5d8bff27bf2, 'name': 'Zii...","{'_id': 5d273ea66497a5d8bff27c07, 'name': 'BAS...","{'_id': 5d273ea66497a5d8bff27c05, 'name': 'Fuz...","{'_id': 5d273ea66497a5d8bff27bf7, 'name': 'vuf...","{'_id': 5d273ea66497a5d8bff27bf9, 'name': 'vuf...","{'_id': 5d273ea66497a5d8bff27c09, 'name': 'Rip...","{'_id': 5d273ea66497a5d8bff27c08, 'name': 'Fli...",...,"{'_id': 5d273ea66497a5d8bff27bf6, 'name': 'Int...","{'_id': 5d273ea66497a5d8bff27bf8, 'name': 'Int...","{'_id': 5d273ea66497a5d8bff27bfe, 'name': 'Yip...","{'_id': 5d273ea66497a5d8bff27bff, 'name': 'Uni...","{'_id': 5d273ea66497a5d8bff27c01, 'name': 'Rea...","{'_id': 5d273ea66497a5d8bff27c04, 'name': 'Clo...","{'_id': 5d273ea66497a5d8bff27bfa, 'name': 'Nav...","{'_id': 5d273ea66497a5d8bff27bf3, 'name': 'hea...","{'_id': 5d273ea66497a5d8bff27c06, 'name': 'Ten...","{'_id': 5d273ea66497a5d8bff27c0b, 'name': 'eki..."
3,"{'_id': 5d273ea66497a5d8bff27bf1, 'name': 'Tit...","{'_id': 5d273ea66497a5d8bff27bf5, 'name': 'Pix...","{'_id': 5d273ea66497a5d8bff27c00, 'name': 'Ind...","{'_id': 5d273ea66497a5d8bff27c08, 'name': 'Fli...","{'_id': 5d273ea66497a5d8bff27c05, 'name': 'Fuz...","{'_id': 5d273ea66497a5d8bff27c09, 'name': 'Rip...","{'_id': 5d273ea66497a5d8bff27c07, 'name': 'BAS...","{'_id': 5d273ea66497a5d8bff27bf0, 'name': 'Fix...","{'_id': 5d273ea66497a5d8bff27bfb, 'name': 'Sky...","{'_id': 5d273ea66497a5d8bff27bfd, 'name': 'Soc...",...,"{'_id': 5d273ea66497a5d8bff27bff, 'name': 'Uni...","{'_id': 5d273ea66497a5d8bff27bfe, 'name': 'Yip...","{'_id': 5d273ea66497a5d8bff27c0a, 'name': 'Hay...","{'_id': 5d273ea66497a5d8bff27c02, 'name': 'kin...","{'_id': 5d273ea66497a5d8bff27c01, 'name': 'Rea...","{'_id': 5d273ea66497

In [ ]:

radio_max_meters=2000
data=findNear(b, radio_max_meters)
                                      
d=pd.DataFrame(data)
d.head()

In [ ]:
data1=pd.DataFrame()

# https://developers.google.com/maps/documentation/geocoding/intro
park_avenue_con_21_street = {
    "type":"Point",
    "coordinates":[-73.987308,40.738935]
}
radio_max_meters = 10000
num_offices = findNear(park_avenue_con_21_street, radio_max_meters).count()

print(f"Hay {num_offices} oficinas cerca a {radio_max_meters} metros")
print("La mas cercana es")
print(list(findNear(park_avenue_con_21_street, radio_max_meters).limit(1)))